In [5]:
import re
import weaviate
import numpy as np
import string
import numpy as np
import re
import json
import os
import tarfile
from gensim.models import KeyedVectors


auth_config = weaviate.AuthApiKey(api_key="cSS9XqOYrQ47rPmzCogrKYYm6rDhaZ7DJdCX")

client = weaviate.Client(
    url="https://test-s3sbv82a.weaviate.network",
    auth_client_secret=auth_config
)
paths = ["test-dataset-1.md", "test-dataset-2.md"]
# 腾讯向量词
def extract_tar_gz(file_path, extract_path):
    # 解压缩文件
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=extract_path)

# 检查特定文件是否存在
specific_file_path = 'tencent-ailab-embedding-zh-d100-v0.2.0-s/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt'
tar_gz_path = 'tencent-ailab-embedding-zh-d100-v0.2.0-s.tar.gz'

if not os.path.exists(specific_file_path):
    print("Embedding file not found. Extracting now...")
    extract_tar_gz(tar_gz_path, '')
    print("Extraction complete.")

wv_from_text = KeyedVectors.load_word2vec_format(specific_file_path, binary=False)

client.schema.delete_class("Article")

# 创建类
article_class = {
    "class": "Article",
    "description": "A class to represent articles",
    "properties": [
        {
            "name": "title",
            "description": "The title of the article",
            "dataType": ["text"]
        },
        {
            "name": "author",
            "description": "The author of the article",
            "dataType": ["text"]
        },
        {
            "name": "content",
            "description": "The content of the article",
            "dataType": ["text"]
        }
    ]
}

# 在Weaviate中添加类定义
client.schema.create_class(article_class)
# # get the schema
# schema = client.schema.get()

# # print the schema
# print(json.dumps(schema, indent=4))

In [6]:
# Initialize counter and interval for displaying progress
counter = 0
interval = 200

def process_data(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    title_pattern = r"# (.*?)\n"
    author_pattern = r">\s*(.*?)\n"
    content_pattern = r"(?:>\s*.*?\n)([\s\S]*)"
    title = re.search(title_pattern, content).group(1)
    author = re.search(author_pattern, content).group(1)
    article_content = re.search(content_pattern, content).group(1)
    punctuation = ' '
    article_content = re.sub(f"[{re.escape(punctuation)}]", "", article_content)
    article_content = article_content.replace("\n", "")
    pattern = r'\*\*(.*?)\*\*|。|；'
    segments = re.split(pattern, article_content)
    # 过滤空字符串和None
    segments = [seg.strip() for seg in segments if seg and seg.strip() not in ['##']]
    return title, author, segments

# 把文本分割后添加向量进数据库
def add_article_to_batch(batch, title, author, segment, wv_from_text):
    global counter
    sentence_vector = [wv_from_text[word] for word in segment if word in wv_from_text.key_to_index]
    if sentence_vector:
        sentence_vector_mean = np.mean(sentence_vector, axis=0).tolist()
    else:
        sentence_vector_mean = []
    properties = {
        "title": title,
        "author": author,
        "content": segment.strip()
    }
    batch.add_data_object(properties, 'Article', vector=sentence_vector_mean)
    
    counter += 1
    if counter % interval == 0:
        print(f'Imported {counter} segments...')


# 读取数据
for path in paths:
    title, author, segments = process_data(path)
    client.batch.configure(batch_size=100)

    with client.batch as batch:
        for segment in segments:  
            add_article_to_batch(batch, title, author, segment, wv_from_text)

    print(f'Finished importing {counter} segments.')

def search_Query(keyword):
    keyword = keyword
    keyword_vector = [wv_from_text[word] for word in keyword if word in wv_from_text.key_to_index]
    keyword_vector_mean = np.mean(keyword_vector, axis=0).tolist()

    result = (
        client.query
        .get("Article", ["title", "content","author"])
        .with_hybrid(keyword, alpha=0.5,properties=["content"],vector=keyword_vector_mean)
        .with_limit(5)
        .do()
    )
    print(json.dumps(result, indent=4, ensure_ascii=False))

search_Query('毫无疑问')

Finished importing 45 segments.
Imported 200 segments...
Finished importing 364 segments.
{
    "data": {
        "Get": {
            "Article": [
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "毫无疑问，技术资产是未来支撑具有硬核科创属性和持续高效发展的上市企业市值和商誉的核心资产",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "无畏的执行者则是通过不懈的努力将愿景变成现实",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "约翰·亨尼斯表示创新需要几类人：有远见的预言者、探索者、无畏的执行者。",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "这种生态系统形成了彼此依赖、互补共生、协作繁荣的利益格局，谁也无法卡谁的脖子",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权"